In [1]:
#Importing libraries 
import pandas as pd 
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import cufflinks as cf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D, SimpleRNN
from tensorflow.keras.layers import Embedding

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning:

urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!



In [ ]:
df=pd.read_csv("df_final.csv")
df.drop('Unnamed: 0', axis = 1, inplace = True) #Got an extra unwanted column and removed it
df=df[df['Review_Clean'].notnull()]
df.head()

In [ ]:
X=df['Review_Clean']
Y=df['Reviewer_Score_Cat']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=1000,stratify=Y)
X_train, X_test, Y_train, Y_test

In [ ]:
Xtrain_corpus=[]
for i in X_train:
    Xtrain_corpus.append(i)

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(Xtrain_corpus)        
        
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
Xtest_corpus=[]
for i in X_test:
    Xtest_corpus.append(i)

X_test_counts = count_vect.transform(Xtest_corpus)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

### Logistic Regression Model

In [ ]:
logregmodel = linear_model.LogisticRegression(multi_class='multinomial', solver='lbfgs')
logregmodel.fit(X_train_tfidf, Y_train)
pred=logregmodel.predict(X_train_tfidf)

In [ ]:
cm = confusion_matrix(Y_train, pred)
print('Logistic Regression Train Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm)) 
print(accuracy)

In [ ]:
pred=logregmodel.predict(X_test_tfidf)
cm = confusion_matrix(Y_test, pred)
print('Logistic Regression Test Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

In [ ]:
#Tune Penalty
scores=[]
labels=[]
for i in [0,0.0001,0.001,0.01,0.1,1]:
    logregmodel=linear_model.LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=i)
    cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    score=cross_val_score(logregmodel, X_train_tfidf, Y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    scores.append(score)
    labels.append(i)

In [ ]:
plt.boxplot(scores, labels=labels, showmeans=True)
plt.show()
#C=1 yields the best result and hence the model remains

### Decision Tree Model

In [ ]:
dtmodel = DecisionTreeClassifier()
dtmodel.fit(X_train_tfidf,Y_train)

In [ ]:
pred = dtmodel.predict(X_train_tfidf)
cm = confusion_matrix(Y_train, pred)
print('Decision Tree Train Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm)) 
print(accuracy)

In [ ]:
pred = dtmodel.predict(X_test_tfidf)
cm = confusion_matrix(Y_test, pred)
print('Decision Tree Test Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm)) 
print(accuracy)

#Clearly, the model has overfit the data

In [ ]:
dtmodel.get_depth() #No wonder it overfit

In [ ]:
#Optimising hyperparameters for the decision tree model

var_max_depth = []
var_criterion = []
var_ccp_alpha = []
trainset_accuracy = []
testset_accuracy = []

for i in range(5, 150, 3):
    for j in ['gini', 'entropy']:
        for k in [0, 0.0001, 0.001, 0.01, 0.1, 1]:
            model = DecisionTreeClassifier(max_depth = i, criterion = j, ccp_alpha = k)
            m1 = model.fit(X_train_tfidf,Y_train)
            pred = m1.predict(X_train_tfidf)
            cm = confusion_matrix(Y_train, pred)
            print("Parameters: ", i, j, k)
            print('Decision Tree Train Accuracy')
            accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm)) 
            print(accuracy)
            trainset_accuracy.append(accuracy)

            pred = m1.predict(X_test_tfidf)
            cm = confusion_matrix(Y_test, pred)
            print('Decision Tree Test Accuracy')
            accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm)) 
            print(accuracy)
            print("=========")
            print()
            testset_accuracy.append(accuracy)
            
            var_max_depth.append(i)
            var_criterion.append(j)
            var_ccp_alpha.append(k)

In [ ]:
df_dt = pd.DataFrame(list(zip(var_max_depth, var_criterion, var_ccp_alpha, trainset_accuracy, testset_accuracy)), columns = ['Max_Depth', 'Criterion', 'CCP_Alpha', 'Trainset_Acc', 'Testset_Acc'])
df_dt['Difference'] = abs(df_dt['Trainset_Acc'] - df_dt['Testset_Acc'])
df_dt[df_dt.Testset_Acc == df_dt.Testset_Acc.max()]

#We can choose the model that gives us the lowest difference betwene trainset and testset accuracies (As it is generalising to unseen data better), and within it, the highest accuracy figures
#We can see that this yeilds an optimal model with multiple max_depth levels (As they all provide same accuracy metrics)
#Taking the lowest depth value, we get a model with a max_depth of 23 and testset accuracy of 0.592516

### Multinomial Naive Bayes

In [ ]:
nbmodel=MultinomialNB()
nbmodel.fit(X_train_tfidf, Y_train)
pred = nbmodel.predict(X_train_tfidf)
cm = confusion_matrix(Y_train, pred)
print('Naives Bayes Multinomial Train Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

In [ ]:
pred = nbmodel.predict(X_test_tfidf)
cm = confusion_matrix(Y_test, pred)
print('Naives Bayes Multinomial Test Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

### XG Boost

In [ ]:
xgbmodel=GradientBoostingClassifier()
xgbmodel.fit(X_train_tfidf, Y_train)
pred = xgbmodel.predict(X_train_tfidf)
cm=confusion_matrix(Y_train, pred)
print('XGBoost Train Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

In [ ]:
pred=xgbmodel.predict(X_test_tfidf)
cm=confusion_matrix(Y_test, pred)
print('XGBoost Test Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

In [ ]:
#Optimisation
X_train1,X_validate,Y_train1,Y_validate=train_test_split(X_train_tfidf,Y_train,test_size=0.2,random_state=1000,stratify=Y_train)
X_train1,X_validate,Y_train1,Y_validate

In [ ]:
#Optimisation
for i in range(20):
    print(i)
    model = GradientBoostingClassifier(max_depth=i+1)
    model.fit(X_train1, Y_train1)
    pred = model.predict(X_validate)
    cm = confusion_matrix(Y_validate, pred)
    accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
    print(accuracy)
    print("===")

In [ ]:
xgbmodel=GradientBoostingClassifier(max_depth=14)
xgbmodel.fit(X_train_tfidf, Y_train)
pred = xgbmodel.predict(X_train_tfidf)
cm=confusion_matrix(Y_train, pred)
print('XGBoost Train Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

In [ ]:
pred=xgbmodel.predict(X_test_tfidf)
cm=confusion_matrix(Y_test, pred)
print('XGBoost Test Accuracy')
print(cm)
accuracy = (cm[0,0]+cm[1,1]+cm[2,2])/sum(sum(cm))
print(accuracy)

### Artificial Neural Network

In [ ]:
# Set random seed
tf.random.set_seed(42)


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(3, activation = 'sigmoid')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Create a learning rate scheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/10)) # traverse a set of learning rate values starting from 1e-4, increasing by 10**(epoch/2) every 10 epoch

print(model.summary())

In [ ]:
history = model.fit(X_train_tfidf.toarray(),Y_train, epochs=50, callbacks=[lr_scheduler])

In [ ]:
model.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Checkout the history
pd.DataFrame(history.history).plot(figsize=(10,7));

In [ ]:
# Plot the learning rate versus the loss
lrs = 1e-4 * (10 ** (np.arange(50)/10))
plt.figure(figsize=(10, 7))
plt.semilogx(lrs, history.history["loss"]) # we want the x-axis (learning rate) to be log scale
plt.xlabel("Learning Rate")
plt.ylabel("Loss")
plt.title("Learning rate vs. loss");

As seen in this graph, loss of the NN was the lowest when the learning rate was between the 10^-3 to 10^-1 range

We will use a learning rate of 10^-3 as a large learning rate of 10^-2  can cause the model to converge too quickly to a suboptimal solution

#### Modified NN Iteration

In [ ]:
# Set random seed
tf.random.set_seed(42)


model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model_2.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

print(model_2.summary())

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_2 = model_2.fit(X_train_tfidf.toarray(),Y_train,batch_size = 5, epochs = 50)

In [ ]:
model_2.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Checkout the history
pd.DataFrame(history_2.history).plot(figsize=(10,7));

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_3 = model_2.fit(X_train_tfidf.toarray(),Y_train,batch_size = 5, epochs = 30)

In [ ]:
model_2.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Set random seed
tf.random.set_seed(42)


model_3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model_3.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.01),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

print(model_3.summary())

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_4 = model_3.fit(X_train_tfidf.toarray(),Y_train,batch_size = 5, epochs = 50)

In [ ]:
model_3.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Set random seed
tf.random.set_seed(42)


model_4 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model_4.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

print(model_4.summary())

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_4 = model_4.fit(X_train_tfidf.toarray(),Y_train, batch_size = 5, epochs = 50)

In [ ]:
model_4.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_6 = model_4.fit(X_train_tfidf.toarray(),Y_train, batch_size = 10, epochs = 50)

In [ ]:
model_4.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_7 = model_4.fit(X_train_tfidf.toarray(),Y_train, batch_size = 5, epochs = 30)

In [ ]:
model_4.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_7 = model_4.fit(X_train_tfidf.toarray(),Y_train, batch_size = 10, epochs = 30)

In [ ]:
model_4.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Set random seed
tf.random.set_seed(42)


model_5 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model_5.compile(optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

print(model_5.summary())

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_8 = model_5.fit(X_train_tfidf.toarray(),Y_train, batch_size = 5, epochs = 50)

In [ ]:
model_5.evaluate(X_test_tfidf.toarray(), Y_test)

In [ ]:
# Set random seed
tf.random.set_seed(42)


model_6 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1, 15247)), #input shape = 1,15247
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='sigmoid'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation = 'sigmoid'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model_6.compile(optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

print(model_6.summary())

In [ ]:
# Fit the model with lower number of epoches based on the loss and accuracy curves of previous graph
history_9 = model_6.fit(X_train_tfidf.toarray(),Y_train, batch_size = 5, epochs = 50)

In [ ]:
model_6.evaluate(X_test_tfidf.toarray(), Y_test)

We see that models 5 and 6 perform best; we can select model 6 as the 'ideal' ANN model, as it offers a low difference between trainset and testset accuracies, and offers a reasonably high testset accuracy of 63.5%